# Notebook for visualizing grids scheme on map

In [1]:
import os
import sys

def find_project_root(target_folder="masteroppgave"):
    """Find the absolute path of a folder (e.g., 'masteroppgave') by searching upward."""
    currentdir = os.path.abspath("__file__")  # Get absolute script path
    while True:
        if os.path.basename(currentdir) == target_folder:
            return currentdir  # Found the target folder
        parentdir = os.path.dirname(currentdir)
        if parentdir == currentdir:  # Stop at filesystem root
            return None
        currentdir = parentdir  # Move one level up

# Find and append 'masteroppgave' to sys.path dynamically
project_root = find_project_root("masteroppgave")

if project_root:
    sys.path.append(project_root)
    print(f"Project root found: {project_root}")
else:
    raise RuntimeError("Could not find 'masteroppgave' directory")

Project root found: /Users/thomasnitsche/Developer/skole/master/masteroppgave


In [2]:
import folium
from folium.plugins import FeatureGroupSubGroup
import random
from schemes.lsh_grid import GridLSH
from constants import  *
from utils.helpers import file_handler
import pprint

## Rome

In [3]:
ROME_DATA_FOLDER = "../dataset/rome/output/"

### Define Grid

In [4]:
Grid = GridLSH(
        "G1",
        min_lat=R_MIN_LAT,
        max_lat=R_MAX_LAT,
        min_lon=R_MIN_LON,
        max_lon=R_MAX_LON,
        resolution=0.2,
        layers=2,
        meta_file="meta.txt",
        data_path="/data",
    )
print(Grid)

Grid: G1
Covering: (5.559754011676299, 7.451072531046803) km 
Resolution: 0.2 km 
Distortion: [0.09593547637340981, 0.13654836151517602] km 
Dimensions: (27, 37) cells


### Visualize Grid on map

In [5]:
import folium
from folium.plugins import FeatureGroupSubGroup

def visualize_grid_with_boundary(grid_lsh):
    """
    Visualizes the grid of the GridLSH object using folium and adds a bounding box.
    
    Parameters:
    - grid_lsh (GridLSH): An instance of the GridLSH class.
    
    Returns:
    - A folium map object.
    """
    
    # Define center of the map (average lat/lon)
    center_lat = (grid_lsh.min_lat + grid_lsh.max_lat) / 2
    center_lon = (grid_lsh.min_lon + grid_lsh.max_lon) / 2

    # Initialize folium map
    map_grid = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="OpenStreetMap")

    # Define colors for different layers
    layer_colors = ["red", "blue", "green", "purple", "orange"]

    # Create a base layer group
    base_layer = folium.FeatureGroup(name="Base Map").add_to(map_grid)

    # Add bounding box (dataset boundary)
    folium.Rectangle(
        bounds=[(grid_lsh.min_lat, grid_lsh.min_lon), (grid_lsh.max_lat, grid_lsh.max_lon)],
        color="black",
        weight=2,
        fill=True,
        fill_opacity=0.1,
        popup="Bounding Box"
    ).add_to(base_layer)

    # Iterate over each layer in the grid
    for layer_index, (layer, grid_points) in enumerate(grid_lsh.grid.items()):
        color = layer_colors[layer_index % len(layer_colors)]  # Cycle colors

        # Create a subgroup for each layer
        layer_group = FeatureGroupSubGroup(base_layer, name=f"Layer {layer_index + 1}")
        map_grid.add_child(layer_group)

        latitudes, longitudes = grid_points  # Unpack grid points

        # Draw grid cells as rectangles
        for i in range(len(latitudes) - 1):
            for j in range(len(longitudes) - 1):
                # Define corners of each cell
                top_left = (latitudes[i], longitudes[j])
                bottom_right = (latitudes[i + 1], longitudes[j + 1])

                folium.Rectangle(
                    bounds=[top_left, bottom_right],
                    color=color,
                    fill=True,
                    fill_opacity=0.3,
                    popup=f"Layer {layer_index + 1}\nCell: ({i}, {j})",
                ).add_to(layer_group)

    # Add layer control to toggle between layers
    folium.LayerControl(collapsed=False).add_to(map_grid)

    return map_grid

# Example usage
grid_map = visualize_grid_with_boundary(Grid)
grid_map.save("output/rome/rome_grid_visualization.html")


### Add trajectory

In [6]:
# trajs_1 = file_handler.load_all_trajectory_files(ROME_DATA_FOLDER, "R")
trajs_2 = file_handler.load_trajectory_files(["R_ABA.txt"], ROME_DATA_FOLDER) ## Reads a list of trajectory files, return a dictionary with the filename as key and coordinates as values


../dataset/rome/output/


### Hash trajectory

In [7]:
hash_ = Grid._create_trajectory_hash(trajs_2["R_ABA"])

print(hash_)

[[[41.91704248264849, 12.491831505459803], [41.9188512261144, 12.491831505459803], [41.91704248264849, 12.491831505459803], [41.91704248264849, 12.49424455406097], [41.9188512261144, 12.49424455406097], [41.92065996958031, 12.491831505459803], [41.9188512261144, 12.491831505459803], [41.92065996958031, 12.491831505459803], [41.92065996958031, 12.49424455406097], [41.9188512261144, 12.49424455406097], [41.91704248264849, 12.49424455406097], [41.91704248264849, 12.496657602662134], [41.91704248264849, 12.49424455406097], [41.915233739182575, 12.49424455406097], [41.91342499571666, 12.49424455406097], [41.911616252250745, 12.491831505459803], [41.90980750878483, 12.491831505459803], [41.90980750878483, 12.489418456858637], [41.907998765318915, 12.489418456858637], [41.906190021853, 12.489418456858637], [41.906190021853, 12.491831505459803], [41.904381278387085, 12.491831505459803], [41.904381278387085, 12.49424455406097], [41.90257253492118, 12.491831505459803], [41.90076379145526, 12.489

### Visualize Grid, original trajectory and hashed trajectory on map

In [8]:
import folium
from folium.plugins import FeatureGroupSubGroup

def visualize_grid_with_trajectory(grid_lsh, trajectory, hashed_trajectory):
    """
    Visualizes the grid of the GridLSH object using Folium and adds a single trajectory.

    Parameters:
    - grid_lsh (GridLSH): An instance of the GridLSH class.
    - trajectory (list): List of (lat, lon) coordinates representing a single trajectory.
    - hashed_trajectory (list of lists): Hashed trajectory representation per layer.

    Returns:
    - A Folium map object.
    """

    # Define center of the map (average lat/lon)
    center_lat = (grid_lsh.min_lat + grid_lsh.max_lat) / 2
    center_lon = (grid_lsh.min_lon + grid_lsh.max_lon) / 2

    # Initialize folium map
    map_grid = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="OpenStreetMap")

    # Define colors for different grid layers
    layer_colors = ["red", "blue", "green", "purple", "orange"]

    # Create a base layer group
    base_layer = folium.FeatureGroup(name="Base Map").add_to(map_grid)

    # Add bounding box (dataset boundary)
    folium.Rectangle(
        bounds=[(grid_lsh.min_lat, grid_lsh.min_lon), (grid_lsh.max_lat, grid_lsh.max_lon)],
        color="black",
        weight=2,
        fill=True,
        fill_opacity=0.1,
        popup="Bounding Box"
    ).add_to(base_layer)

    # Iterate over each layer in the grid
    for layer_index, (layer, grid_points) in enumerate(grid_lsh.grid.items()):
        color = layer_colors[layer_index % len(layer_colors)]  # Cycle colors

        # Create a subgroup for each grid layer
        grid_layer = FeatureGroupSubGroup(base_layer, name=f"Grid Layer {layer_index + 1}")
        map_grid.add_child(grid_layer)

        latitudes, longitudes = grid_points  # Unpack grid points

        # Draw grid cells as rectangles
        for i in range(len(latitudes) - 1):
            for j in range(len(longitudes) - 1):
                # Define corners of each cell
                top_left = (latitudes[i], longitudes[j])
                bottom_right = (latitudes[i + 1], longitudes[j + 1])

                folium.Rectangle(
                    bounds=[top_left, bottom_right],
                    color=color,
                    fill=True,
                    fill_opacity=0.3,
                    popup=f"Layer {layer_index + 1}\nCell: ({i}, {j})",
                ).add_to(grid_layer)

    # Plot the original trajectory as a blue polyline
    folium.PolyLine(
        trajectory,
        color="blue",
        weight=2.5,
        opacity=1,
        popup="Trajectory Path"
    ).add_to(base_layer)

    # Add markers for each point along the original trajectory
    for lat, lon in trajectory:
        folium.CircleMarker(
            location=(lat, lon),
            radius=3,  # Small marker
            color="black",
            fill=True,
            fill_opacity=1,
            popup=f"Point: ({lat:.5f}, {lon:.5f})"
        ).add_to(base_layer)
        
    # Plot hashed trajectory points as markers with circles (in the same FeatureGroupSubGroup)
    hashed_layer_colors = ["red", "blue", "green", "purple", "orange"]  # Colors for different layers

    for layer_index, layer_points in enumerate(hashed_trajectory):
        layer_color = hashed_layer_colors[layer_index % len(hashed_layer_colors)]  # Assign color per layer

        # Create a single FeatureGroupSubGroup for both circles and markers
        hashed_layer = FeatureGroupSubGroup(base_layer, name=f"Hashed Points Layer {layer_index + 1}")
        map_grid.add_child(hashed_layer)

        for lat, lon in layer_points:
            # Add a small circle around the hashed point
            folium.Circle(
                location=(lat, lon),
                radius=20,  # Medium-sized circle
                color=layer_color,
                fill=True,
                fill_opacity=0.4,
                popup=f"Layer {layer_index + 1} Hashed Point"
            ).add_to(hashed_layer)

            # Add a marker on top of the hashed point
            folium.Marker(
                location=(lat, lon),
                icon=folium.Icon(color=layer_color, icon="info-sign"),
                popup=f"Hashed Point ({lat:.5f}, {lon:.5f}) - Layer {layer_index + 1}"
            ).add_to(hashed_layer)

    # Add layer control to toggle between layers
    folium.LayerControl(collapsed=False).add_to(map_grid)

    return map_grid

# Generate the map with filtering options
grid_map = visualize_grid_with_trajectory(Grid, trajs_2["R_ABA"], hash_)

# Save the map
grid_map.save("output/rome/rome_grid_with_hashed_trajectory.html")

## Porto

In [9]:
PORTO_DATA_FOLDER = "../dataset/porto/output/"

### Define grid

In [10]:
Grid = GridLSH(
        "G1",
        min_lat=P_MIN_LAT,
        max_lat=P_MAX_LAT,
        min_lon=P_MIN_LON,
        max_lon=P_MAX_LON,
        resolution=0.2,
        layers=2,
        meta_file="meta.txt",
        data_path="/data",
    )
print(Grid)

Grid: G1
Covering: (5.559754011677007, 7.536733782089804) km 
Resolution: 0.2 km 
Distortion: [0.06120570325264674, 0.07243629228034575] km 
Dimensions: (27, 37) cells


### Visualize Grid on map

In [11]:
import folium
from folium.plugins import FeatureGroupSubGroup

def visualize_grid_with_boundary(grid_lsh):
    """
    Visualizes the grid of the GridLSH object using folium and adds a bounding box.
    
    Parameters:
    - grid_lsh (GridLSH): An instance of the GridLSH class.
    
    Returns:
    - A folium map object.
    """
    
    # Define center of the map (average lat/lon)
    center_lat = (grid_lsh.min_lat + grid_lsh.max_lat) / 2
    center_lon = (grid_lsh.min_lon + grid_lsh.max_lon) / 2

    # Initialize folium map
    map_grid = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="OpenStreetMap")

    # Define colors for different layers
    layer_colors = ["red", "blue", "green", "purple", "orange"]

    # Create a base layer group
    base_layer = folium.FeatureGroup(name="Base Map").add_to(map_grid)

    # Add bounding box (dataset boundary)
    folium.Rectangle(
        bounds=[(grid_lsh.min_lat, grid_lsh.min_lon), (grid_lsh.max_lat, grid_lsh.max_lon)],
        color="black",
        weight=2,
        fill=True,
        fill_opacity=0.1,
        popup="Bounding Box"
    ).add_to(base_layer)

    # Iterate over each layer in the grid
    for layer_index, (layer, grid_points) in enumerate(grid_lsh.grid.items()):
        color = layer_colors[layer_index % len(layer_colors)]  # Cycle colors

        # Create a subgroup for each layer
        layer_group = FeatureGroupSubGroup(base_layer, name=f"Layer {layer_index + 1}")
        map_grid.add_child(layer_group)

        latitudes, longitudes = grid_points  # Unpack grid points

        # Draw grid cells as rectangles
        for i in range(len(latitudes) - 1):
            for j in range(len(longitudes) - 1):
                # Define corners of each cell
                top_left = (latitudes[i], longitudes[j])
                bottom_right = (latitudes[i + 1], longitudes[j + 1])

                folium.Rectangle(
                    bounds=[top_left, bottom_right],
                    color=color,
                    fill=True,
                    fill_opacity=0.3,
                    popup=f"Layer {layer_index + 1}\nCell: ({i}, {j})",
                ).add_to(layer_group)

    # Add layer control to toggle between layers
    folium.LayerControl(collapsed=False).add_to(map_grid)

    return map_grid

# Example usage
grid_map = visualize_grid_with_boundary(Grid)
grid_map.save("output/porto/porto_grid_visualization.html")


### Add trajectory

In [12]:
# trajs_1 = file_handler.load_all_trajectory_files(ROME_DATA_FOLDER, "R")
trajs_2 = file_handler.load_trajectory_files(["P_ABA.txt"], PORTO_DATA_FOLDER) ## Reads a list of trajectory files, return a dictionary with the filename as key and coordinates as values

../dataset/porto/output/


### Hash trajectory

In [13]:
hash_ = Grid._create_trajectory_hash(trajs_2["P_ABA"])

print(hash_)

[[[41.15502347480649, -8.613943107867131], [41.153214731340576, -8.613943107867131], [41.153214731340576, -8.611557485568166], [41.153214731340576, -8.6091718632692], [41.15140598787466, -8.6091718632692], [41.15140598787466, -8.611557485568166], [41.149597244408746, -8.611557485568166], [41.149597244408746, -8.6091718632692], [41.14778850094284, -8.6091718632692], [41.14778850094284, -8.606786240970234], [41.14778850094284, -8.604400618671269], [41.14597975747692, -8.604400618671269], [41.14417101401101, -8.604400618671269], [41.14417101401101, -8.606786240970234]], [[41.1551250410791, -8.613809148149057], [41.15331629761319, -8.613809148149057], [41.15331629761319, -8.611423525850093], [41.15331629761319, -8.609037903551126], [41.15331629761319, -8.611423525850093], [41.15331629761319, -8.609037903551126], [41.15150755414727, -8.609037903551126], [41.15150755414727, -8.611423525850093], [41.14969881068136, -8.611423525850093], [41.14969881068136, -8.609037903551126], [41.147890067215

### Visualize Grid, original trajectory and hashed trajectory on map

In [14]:
import folium
from folium.plugins import FeatureGroupSubGroup

def visualize_grid_with_trajectory(grid_lsh, trajectory, hashed_trajectory):
    """
    Visualizes the grid of the GridLSH object using Folium and adds a single trajectory.

    Parameters:
    - grid_lsh (GridLSH): An instance of the GridLSH class.
    - trajectory (list): List of (lat, lon) coordinates representing a single trajectory.
    - hashed_trajectory (list of lists): Hashed trajectory representation per layer.

    Returns:
    - A Folium map object.
    """

    # Define center of the map (average lat/lon)
    center_lat = (grid_lsh.min_lat + grid_lsh.max_lat) / 2
    center_lon = (grid_lsh.min_lon + grid_lsh.max_lon) / 2

    # Initialize folium map
    map_grid = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="OpenStreetMap")

    # Define colors for different grid layers
    layer_colors = ["red", "blue", "green", "purple", "orange"]

    # Create a base layer group
    base_layer = folium.FeatureGroup(name="Base Map").add_to(map_grid)

    # Add bounding box (dataset boundary)
    folium.Rectangle(
        bounds=[(grid_lsh.min_lat, grid_lsh.min_lon), (grid_lsh.max_lat, grid_lsh.max_lon)],
        color="black",
        weight=2,
        fill=True,
        fill_opacity=0.1,
        popup="Bounding Box"
    ).add_to(base_layer)

    # Iterate over each layer in the grid
    for layer_index, (layer, grid_points) in enumerate(grid_lsh.grid.items()):
        color = layer_colors[layer_index % len(layer_colors)]  # Cycle colors

        # Create a subgroup for each grid layer
        grid_layer = FeatureGroupSubGroup(base_layer, name=f"Grid Layer {layer_index + 1}")
        map_grid.add_child(grid_layer)

        latitudes, longitudes = grid_points  # Unpack grid points

        # Draw grid cells as rectangles
        for i in range(len(latitudes) - 1):
            for j in range(len(longitudes) - 1):
                # Define corners of each cell
                top_left = (latitudes[i], longitudes[j])
                bottom_right = (latitudes[i + 1], longitudes[j + 1])

                folium.Rectangle(
                    bounds=[top_left, bottom_right],
                    color=color,
                    fill=True,
                    fill_opacity=0.3,
                    popup=f"Layer {layer_index + 1}\nCell: ({i}, {j})",
                ).add_to(grid_layer)

    # Plot the original trajectory as a blue polyline
    folium.PolyLine(
        trajectory,
        color="blue",
        weight=2.5,
        opacity=1,
        popup="Trajectory Path"
    ).add_to(base_layer)

    # Add markers for each point along the original trajectory
    for lat, lon in trajectory:
        folium.CircleMarker(
            location=(lat, lon),
            radius=3,  # Small marker
            color="black",
            fill=True,
            fill_opacity=1,
            popup=f"Point: ({lat:.5f}, {lon:.5f})"
        ).add_to(base_layer)
        
    # Plot hashed trajectory points as markers with circles (in the same FeatureGroupSubGroup)
    hashed_layer_colors = ["red", "blue", "green", "purple", "orange"]  # Colors for different layers

    for layer_index, layer_points in enumerate(hashed_trajectory):
        layer_color = hashed_layer_colors[layer_index % len(hashed_layer_colors)]  # Assign color per layer

        # Create a single FeatureGroupSubGroup for both circles and markers
        hashed_layer = FeatureGroupSubGroup(base_layer, name=f"Hashed Points Layer {layer_index + 1}")
        map_grid.add_child(hashed_layer)

        for lat, lon in layer_points:
            # Add a small circle around the hashed point
            folium.Circle(
                location=(lat, lon),
                radius=20,  # Medium-sized circle
                color=layer_color,
                fill=True,
                fill_opacity=0.4,
                popup=f"Layer {layer_index + 1} Hashed Point"
            ).add_to(hashed_layer)

            # Add a marker on top of the hashed point
            folium.Marker(
                location=(lat, lon),
                icon=folium.Icon(color=layer_color, icon="info-sign"),
                popup=f"Hashed Point ({lat:.5f}, {lon:.5f}) - Layer {layer_index + 1}"
            ).add_to(hashed_layer)

    # Add layer control to toggle between layers
    folium.LayerControl(collapsed=False).add_to(map_grid)

    return map_grid

# Generate the map with filtering options
grid_map = visualize_grid_with_trajectory(Grid, trajs_2["P_ABA"], hash_)

# Save the map
grid_map.save("output/porto/porto_grid_with_hashed_trajectory.html")